In [3]:
import os
import cv2 as cv
import numpy as np
import random

# Anisotropic Diffusion Filter function
def anisodiff_color(img, niter=1, kappa=50, gamma=0.1, step=(1., 1.), option=1):
    img = img.astype('float32')
    imgout = img.copy()
    deltaS = np.zeros_like(imgout)
    deltaE = deltaS.copy()
    NS = deltaS.copy()
    EW = deltaS.copy()
    gS = np.ones_like(imgout)
    gE = gS.copy()

    for ii in range(niter):
        deltaS[:-1, :] = np.diff(imgout, axis=0)
        deltaE[:, :-1] = np.diff(imgout, axis=1)
        if option == 1:
            gS = np.exp(-(deltaS/kappa)**2.)/step[0]
            gE = np.exp(-(deltaE/kappa)**2.)/step[1]
        elif option == 2:
            gS = 1./(1.+(deltaS/kappa)**2.)/step[0]
            gE = 1./(1.+(deltaE/kappa)**2.)/step[1]
        E = gE * deltaE
        S = gS * deltaS
        NS[:] = S
        EW[:] = E
        NS[1:, :] -= S[:-1, :]
        EW[:, 1:] -= E[:, :-1]
        imgout += gamma * (NS + EW)

    return imgout

# Directories setup
input_dir = './demo_datasets_week8'  # Your input dataset directory
output_dir = './Demo_Dataset(anisotropic+flip)'  # Directory for processed images

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define supported image extensions including .tif and .tiff
supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')

# Process each class folder and each image
for class_folder in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_folder)
    if not os.path.isdir(class_path):
        continue
    
    # Create corresponding class folder in the output directory
    output_class_path = os.path.join(output_dir, class_folder)
    if not os.path.exists(output_class_path):
        os.makedirs(output_class_path)
    
    # Process each image in the class folder
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        
        # Check if the file is an image by extension
        if not image_name.lower().endswith(supported_extensions):
            print(f"Skipping non-image file: {image_path}")
            continue
        
        # Load the image
        image = cv.imread(image_path, cv.IMREAD_UNCHANGED)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue
        
        # Convert to RGB if not already in correct format
        if len(image.shape) == 2 or image.shape[2] == 1:  # Handle grayscale images
            image = cv.cvtColor(image, cv.COLOR_GRAY2RGB)
        elif image.shape[2] == 4:  # Handle images with alpha channel
            image = cv.cvtColor(image, cv.COLOR_BGRA2RGB)
        else:
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        
        # Apply Anisotropic Diffusion
        diffused_image = anisodiff_color(image, niter=10, kappa=100, gamma=0.1, step=(1., 1.), option=1)
        
        # Rescale to [0, 255] and ensure type is uint8
        diffused_image = np.clip(diffused_image, 0, 255).astype(np.uint8)
        
        # Convert back to BGR for saving
        diffused_image_bgr = cv.cvtColor(diffused_image, cv.COLOR_RGB2BGR)

        # Randomly decide to apply a horizontal flip
        if random.random() < 0.1:
            final_image = cv.flip(diffused_image_bgr, 1)  # Mirror flip horizontally
        else:
            final_image = diffused_image_bgr
        
        # Save the processed image as JPEG or PNG
        output_image_path = os.path.join(output_class_path, image_name.replace('.tif', '.png'))  # Save as PNG or JPEG
        cv.imwrite(output_image_path, final_image)
        
        print(f"Processed and saved: {output_image_path}")


Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\1015_CRC-Prim-HE-10_020.png_Row_601_Col_601.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\1040C_CRC-Prim-HE-05_032.png_Row_1051_Col_2101.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\1050A_CRC-Prim-HE-03_012.png_Row_301_Col_1351.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\109EF_CRC-Prim-HE-03_012.png_Row_3151_Col_901.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\10C21_CRC-Prim-HE-05_032.png_Row_1201_Col_3901.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\110E3_CRC-Prim-HE-07.png_Row_301_Col_3901.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\11472_CRC-Prim-HE-07.png_Row_1201_Col_2701.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\1157_CRC-Prim-HE-05_032.png_Row_1051_Col_2701.png
Processed and saved: ./Demo_Dataset(anisotropic+flip)\ADI\11744_CRC-Prim-HE-07.png_Row_1_Col_3301.png
Processed and saved: ./Demo_Dataset(aniso

In [5]:
import os

# Define the path to the processed dataset directory
output_dir = './Demo_Dataset(anisotropic+flip)'

# Initialize a dictionary to hold the counts for each class
class_counts = {}

# Iterate through each class folder in the output directory
for class_folder in os.listdir(output_dir):
    class_path = os.path.join(output_dir, class_folder)
    
    # Check if it is a directory
    if os.path.isdir(class_path):
        # Count the number of image files in the class folder
        num_images = len([img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'))])
        class_counts[class_folder] = num_images

# Print the counts for each class
for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} images")


Class 'ADI': 1000 images
Class 'DEBRIS_MUCUS': 1000 images
Class 'LYM': 1000 images
Class 'MUSC_STROMA': 1000 images
Class 'NORM': 1000 images
Class 'TUM': 1000 images
